<h2>Project 3: Na&iuml;ve Bayes and the Perceptron</h2>

<blockquote>
    <center>
    <img src="nb.png" width="200px" />
    </center>
      <p><cite><center>"All models are wrong, but some are useful."<br>
       -- George E.P. Box
      </center></cite></p>
</blockquote>

<h3>Introduction</h3>
<!--Aðalbrandr-->

<p>A&eth;albrandr is visiting America from Norway and has been having the hardest time distinguishing boys and girls because of the weird American names like Jack and Jane.  This has been causing lots of problems for A&eth;albrandr when he goes on dates. When he heard that Cornell has a Machine Learning class, he asked that we help him identify the gender of a person based on their name to the best of our ability.  In this project, you will implement Na&iuml;ve Bayes to predict if a name is male or female. </p>

<strong>How to submit:</strong> You can submit your code using the red <strong>Submit</strong> button above. This button will send any code below surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags below to the autograder, which will then run several tests over your code. By clicking on the <strong>Details</strong> dropdown next to the Submit button, you will be able to view your submission report once the autograder has completed running. This submission report contains a summary of the tests you have failed or passed, as well as a log of any errors generated by your code when we ran it.

Note that this may take a while depending on how long your code takes to run! Once your code is submitted you may navigate away from the page as you desire -- the most recent submission report will always be available from the Details menu.

<p><strong>Evaluation:</strong> Your code will be autograded for technical
correctness and--on some assignments--speed. Please <em>do not</em> change the names of any provided functions or classes within the code, or you will wreak havoc on the autograder. Furthermore, <em>any code not surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags will not be run by the autograder</em>. However, the correctness of your implementation -- not the autograder's output -- will be the final judge of your score.  If necessary, we will review and grade assignments individually to ensure that you receive due credit for your work.

<p><strong>Academic Integrity:</strong> We will be checking your code against other submissions in the class for logical redundancy. If you copy someone else's code and submit it with minor changes, we will know. These cheat detectors are quite hard to fool, so please don't try. We trust you all to submit your own work only; <em>please</em> don't let us down. If you do, we will pursue the strongest consequences available to us.

<p><strong>Getting Help:</strong> You are not alone!  If you find yourself stuck  on something, contact the course staff for help.  Office hours, section, and the <a href="https://piazza.com/class/iyag4nk2rsxsv">Piazza</a> are there for your support; please use them.  If you can't make our office hours, let us know and we will schedule more.  We want these projects to be rewarding and instructional, not frustrating and demoralizing.  But, we don't know when or how to help unless you ask.
  

<h3> Of boys and girls </h3>

<p> Take a look at the files <code>girls.train</code> and <code>boys.train</code>. For example with the unix command <pre>cat girls.train</pre> 
<pre>
...
Addisyn
Danika
Emilee
Aurora
Julianna
Sophia
Kaylyn
Litzy
Hadassah
</pre>
Believe it or not, these are all more or less common girl names. The problem with the current file is that the names are in plain text, which makes it hard for a machine learning algorithm to do anything useful with them. We therefore need to transform them into some vector format, where each name becomes a vector that represents a point in some high dimensional input space. </p>

<p>That is exactly what the following Python function <code>name2features</code> does: </p>

In [1]:
#<GRADED>
import numpy as np
#</GRADED>
import sys

# add p03 folder
sys.path.insert(0, './p03/')

%matplotlib inline

In [2]:
def hashfeatures(baby, B, FIX):
    v = np.zeros(B)
    for m in range(FIX):
        featurestring = "prefix" + baby[:m]
        v[hash(featurestring) % B] = 1
        featurestring = "suffix" + baby[-1*m:]
        v[hash(featurestring) % B] = 1
    return v

In [3]:
def name2features(filename, B=128, FIX=3, LoadFile=True):
    """
    Output:
    X : n feature vectors of dimension B, (nxB)
    """
    # read in baby names
    if LoadFile:
        with open(filename, 'r') as f:
            babynames = [x.rstrip() for x in f.readlines() if len(x) > 0]
    else:
        babynames = filename.split('\n')
    n = len(babynames)
    X = np.zeros((n, B))
    for i in range(n):
        X[i,:] = hashfeatures(babynames[i], B, FIX)
    return X

In [4]:
#workarea
bname='Laura'
hashfeatures(bname,128,3)
print(bname[:3])
print(bname[-1*3:])
fs = "prefix"+bname[:0]
hash(fs )% 128


Lau
ura


34

It reads every name in the given file and converts it into a 128-dimensional feature vector. </p> 

<p>Can you figure out what the features are? (Understanding how these features are constructed will help you later on in the competition.)<br></p>

<p>We have provided you with a python function <code>genTrainFeatures</code>, which calls this script, transforms the names into features and loads them into memory. 

In [5]:
def genTrainFeatures(dimension=128, fix=3):
    """
    function [x,y]=genTrainFeatures
    
    This function calls the python script "name2features.py" 
    to convert names into feature vectors and loads in the training data. 
    
    
    Output: 
    x: n feature vectors of dimensionality d [d,n]
    y: n labels (-1 = girl, +1 = boy)
    """
    
    # Load in the data
    Xgirls = name2features("girls.train", B=dimension, FIX=fix)
    Xboys = name2features("boys.train", B=dimension, FIX=fix)
    X = np.concatenate([Xgirls, Xboys])
    
    # Generate Labels
    Y = np.concatenate([-np.ones(len(Xgirls)), np.ones(len(Xboys))])
    
    # shuffle data into random order
    ii = np.random.permutation([i for i in range(len(Y))])
    
    return X[ii, :], Y[ii]

You can call the following command to load in the features and the labels of all boys and girls names. 

In [6]:
X,Y = genTrainFeatures(128)

In [7]:
#workarea
print(X[1,:])
y=[1,1,-1,-1,2]
[label for label in y if label==1]
np.ones((2,12))
indices=[i for i, inp_label in enumerate(y) if inp_label == -1]

a=np.matrix([[1,2,1,-1],[2,-1,4,2],[1,1,3,1],[-1,-1,4,2]])
y1=[-1,1,1,-1]
ni=[i for i, inp_label in enumerate(y1) if inp_label == -1]
a[ni,:]
print(a)
print(a.sum())
print(np.einsum('ij->',a))
#print(np.log(a))
p=[3,2,5,2]
q=[7,8,3,1]
print(np.log(p))
print(np.log(q))
print(np.log(p) - np.log(q))


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0.]
[[ 1  2  1 -1]
 [ 2 -1  4  2]
 [ 1  1  3  1]
 [-1 -1  4  2]]
20
20
[1.09861229 0.69314718 1.60943791 0.69314718]
[1.94591015 2.07944154 1.09861229 0.        ]
[-0.84729786 -1.38629436  0.51082562  0.69314718]


<h3> The Na&iuml;ve Bayes Classifier </h3>

<p> The Na&iuml;ve Bayes classifier is a linear classifier based on Bayes Rule. The following questions will ask you to finish these functions in a pre-defined order. <br>
<strong>As a general rule, you should avoid tight loops at all cost.</strong></p>
<p>(a) Estimate the class probability P(Y) in 
<b><code>naivebayesPY</code></b>
. This should return the probability that a sample in the training set is positive or negative, independent of its features.
</p>



In [8]:
#<GRADED>
def naivebayesPY(x,y):
    """
    function [pos,neg] = naivebayesPY(x,y);

    Computation of P(Y)
    Input:
        x : n input vectors of d dimensions (nxd)
        y : n labels (-1 or +1) (nx1)

    Output:
    pos: probability p(y=1)
    neg: probability p(y=-1)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    y = np.concatenate([y, [-1,1]])
    n = len(y)
    ## fill in code here
    pos=len([label for label in y if label==1])/n
    neg=len([label for label in y if label==-1])/n
    
    return pos,neg
#</GRADED>

pos,neg = naivebayesPY(X,Y)

<p>(b) Estimate the conditional probabilities P(X|Y) in 
<b><code>naivebayesPXY</code></b>
.  Use a <b>multinomial</b> distribution as model. This will return the probability vectors  for all features given a class label.
</p> 

In [9]:
#<GRADED>
def naivebayesPXY(x,y):
    """
    function [posprob,negprob] = naivebayesPXY(x,y);
    
    Computation of P(X|Y)
    Input:
        x : n input vectors of d dimensions (nxd)
        y : n labels (-1 or +1) (nx1)
    
    Output:
    posprob: probability vector of p(x|y=1) (1xd)
    negprob: probability vector of p(x|y=-1) (1xd)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    n, d = x.shape
    x = np.concatenate([x, np.ones((2,d))])
    y = np.concatenate([y, [-1,1]])
    n, d = x.shape
    
    ## fill in code here
    
    #Get indexes where y=1
    pos_indices = [i for i, inp_label in enumerate(y) if inp_label == 1]
    #Get indexes where y=-1
    neg_indices = [i for i, inp_label in enumerate(y) if inp_label == -1]
    
    #input vectors labelled positive
    inp_pos=x[pos_indices,:]
    
    #input vectore labelled negative
    inp_neg=x[neg_indices,:]
    
    #Initialize output 
    posprob=np.zeros((1,d))
    negprob=np.zeros((1,d))
    
    #Sum over axis=0/individual features
    posprob=(np.einsum('ij->j',inp_pos))/(np.einsum('ij->',inp_pos))
    negprob=(np.einsum('ij->j',inp_neg))/(np.einsum('ij->',inp_neg))
    
    return posprob,negprob
    
#</GRADED>

posprob,negprob = naivebayesPXY(X,Y)

<p>(c) Solve for the log ratio, $\log\left(\frac{P(Y=1 | X = xtest)}{P(Y=-1|X= xtest)}\right)$, using Bayes Rule.
 Implement this in 
<b><code>naivebayes</code></b>.
</p>



In [10]:
#<GRADED>
def naivebayes(x,y,xtest):
    """
    function logratio = naivebayes(x,y);
    
    Computation of log P(Y|X=x1) using Bayes Rule
    Input:
    x : n input vectors of d dimensions (nxd)
    y : n labels (-1 or +1)
    xtest: input vector of d dimensions (1xd)
    
    Output:
    logratio: log (P(Y = 1|X=xtest)/P(Y=-1|X=xtest))
    """
    
    ## fill in code here
    pos,neg = naivebayesPY(x,y)
    posprob,negprob = naivebayesPXY(x,y)
    
    #Get features of test where value=1
    test_feature_indices = [i for i, test_feature in enumerate(xtest) if test_feature >= 1]
    
    
    poslog=np.einsum('i->',np.log(posprob[test_feature_indices]))+ np.log(pos)
    neglog=np.einsum('i->',np.log(negprob[test_feature_indices]))+ np.log(neg)
    
    return poslog-neglog

#</GRADED>

p = naivebayes(X,Y,X[0,:])

<p>(d) Naïve Bayes can also be written as a linear classifier. Implement this in 
<b><code>naivebayesCL</code></b>.

In [11]:
#<GRADED>
def naivebayesCL(x,y):
    """
    function [w,b]=naivebayesCL(x,y);
    Implementation of a Naive Bayes classifier
    Input:
    x : n input vectors of d dimensions (nxd)
    y : n labels (-1 or +1)

    Output:
    w : weight vector of d dimensions
    b : bias (scalar)
    """
    
    n, d = x.shape
    ## fill in code here
    pos,neg = naivebayesPY(x,y)
    posprob,negprob = naivebayesPXY(x,y)
    
    #Initialize output
    w=np.zeros((1,d))
    w=np.log(posprob)-np.log(negprob)
    b=np.log(pos)-np.log(neg)
    
    return w,b

#</GRADED>

w,b = naivebayesCL(X,Y)

<p>(e) Implement 
<b><code>classifyLinear</code></b>
 that applies a linear weight vector and bias to a set of input vectors and outputs their predictions.  (You can use your answer from the previous project.)
 
 

In [12]:
#<GRADED>
def innerproduct(X,Z=None):
    # function innerproduct(X,Z)
    #
    # Computes the inner-product matrix.
    # Syntax:
    # D=innerproduct(X,Z)
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix G of size nxm
    # G[i,j] is the inner-product between vectors X[i,:] and Z[j,:]
    #
    # call with only one input:
    # innerproduct(X)=innerproduct(X,X)
    #
    if Z is None: # case when there is only one input (X)
        G=np.matmul(X,X.transpose())
    else:  # case when there are two inputs (X,Z)
        G=np.matmul(X,Z.transpose())
    
    return G
#</GRADED>


In [13]:
#<GRADED>
def classifyLinear(x,w,b=0):
    """
    function preds=classifyLinear(x,w,b);
    
    Make predictions with a linear classifier
    Input:
    x : n input vectors of d dimensions (nxd)
    w : weight vector of d dimensions
    b : bias (optional)
    
    Output:
    preds: predictions
    """
    
    ## fill in code here


    w = w.reshape(1,-1)
    
    d=w.shape[1]
    x=x.reshape(-1,d)
    n=x.shape[0]
    preds=np.zeros(n)
    
    for i in range(n):
        if innerproduct(w,x[i,:])+b>=0:
            preds[i]=1
        else:
            preds[i]=-1
    
    return preds


#</GRADED>

print('Training error: %.2f%%' % (100 *(classifyLinear(X, w, b) != Y).mean()))

Training error: 22.92%


You can now test your code with the following interactive name classification script:

In [14]:
DIMS = 128
print('Loading data ...')
X,Y = genTrainFeatures(DIMS)
print('Training classifier ...')
w,b=naivebayesCL(X,Y)
error = np.mean(classifyLinear(X,w,b) != Y)
print('Training error: %.2f%%' % (100 * error))

while True:
    print('Please enter your name>')
    yourname = input()
    if len(yourname) < 1:
        break
    xtest = name2features(yourname,B=DIMS,LoadFile=False)
    pred = classifyLinear(xtest,w,b)[0]
    if pred > 0:
        print("%s, I am sure you are a nice boy.\n" % yourname)
    else:
        print("%s, I am sure you are a nice girl.\n" % yourname)

Loading data ...
Training classifier ...
Training error: 22.92%
Please enter your name>



In [15]:
#List few misclassified samples
preds=classifyLinear(X,w,b)
indices=[i for i,x in enumerate(classifyLinear(X,w,b) != Y)  if x]
indices



[17,
 19,
 22,
 23,
 32,
 33,
 39,
 40,
 44,
 46,
 47,
 52,
 55,
 65,
 66,
 72,
 76,
 79,
 82,
 83,
 87,
 93,
 103,
 124,
 125,
 132,
 134,
 135,
 136,
 147,
 150,
 152,
 155,
 170,
 181,
 190,
 194,
 200,
 203,
 205,
 209,
 210,
 212,
 218,
 221,
 224,
 225,
 230,
 232,
 233,
 234,
 237,
 242,
 246,
 250,
 251,
 252,
 254,
 259,
 267,
 273,
 278,
 287,
 296,
 301,
 304,
 305,
 320,
 329,
 330,
 339,
 340,
 341,
 346,
 349,
 351,
 360,
 362,
 364,
 368,
 370,
 379,
 381,
 385,
 394,
 398,
 399,
 408,
 413,
 419,
 424,
 426,
 429,
 432,
 434,
 441,
 443,
 444,
 451,
 457,
 464,
 472,
 473,
 474,
 476,
 478,
 489,
 493,
 496,
 497,
 503,
 505,
 506,
 507,
 510,
 515,
 527,
 529,
 536,
 539,
 543,
 544,
 567,
 570,
 572,
 581,
 582,
 583,
 588,
 589,
 591,
 592,
 599,
 603,
 607,
 608,
 609,
 614,
 615,
 616,
 618,
 621,
 624,
 626,
 632,
 646,
 647,
 657,
 662,
 663,
 668,
 674,
 679,
 680,
 681,
 687,
 695,
 702,
 711,
 717,
 721,
 731,
 734,
 736,
 739,
 743,
 747,
 749,
 754,
 757,
 7

<h3> Feature Extraction (Competition)</h3>

<p>(e) (<b>optional</b>) As always, this programming project also includes a competition.  We will rank all submissions by how well your Na&iuml;ve Bayes classifier performs on a secret test set. If you want to improve your classifier modify <code>name2features2</code> below.   The automatic reader will use your Python script to extract features and train your classifier on the same names training set by calling the function with only one argument--the name of a file containing a list of names.  The given implementation is the same as the given <code>name2features</code> above.
</p>
  

In [1]:
#<GRADED>
def hashfeatures(baby, B, FIX):
    v = np.zeros(B)
    #nef - number of extra features
    nef=0
    baby=baby.lower()
    
    for m in range(FIX):
        featurestring = "prefix" + baby[:m]
        v[hash(featurestring) % B-nef] = 1
        featurestring = "suffix" + baby[-1*m:]
        v[hash(featurestring) % B-nef] = 1
        
    for m in range(1,len(baby)):
        if baby[m-1] == baby[m]:
            v[hash("repetition"+ baby[m]+baby[m]) % B-nef] = 1
 
    featurestring = "vowel" + ("true" if baby[-1] in "aeiouy"  else "false")
    v[hash(featurestring) % B-nef] = 1
    featurestring = "vowelnumber" + ("true" if np.sum(list(map(baby.count, "aeiou")))>=3 else "false")
    v[hash(featurestring) % B-nef] = 1
#     featurestring = "twovowels" + ("true" if baby[-1] in "aeiou" and baby[-2] in "aeiou"  else "false")
#     v[hash(featurestring) % B-nef] = 1 
#     middle = int(len(baby)/2)
#     featurestring = "middle" + baby[middle]
#     v[hash(featurestring) % B-nef] = 1
#     featurestring = "middle" + baby[middle-1] + baby[middle]
#     v[hash(featurestring) % B-nef] = 1
    
        #How to avoid collision? Avoiding collision increses accuracy tried(100000) 0.799 acc
        
        #improve hash function
        #feature extraction
        #hyperparam tuning
        #dimension redn
        
        #sparse dataset is always a problem
        #most significant letters
        
        #fill up the matrix more
    
    #Taking first two and second two
    #featurestring = "prefix" + baby[:2]
    #v[hash(featurestring) % B-nef] = 2
    #featurestring = "prefix" + baby[1:3]
    #v[hash(featurestring) % B-nef] = 2
    
    
    
    #Check the last char - vowel or y
    #v[B-nef]= 2 if baby[-1] in "aeiouy"  else 0
        
    #Check if last two letters are vowels
    #v[B-nef+1]= 2 if baby[-1] in "aeiou" and baby[-2] in "aeiou" else 0

    #Check if last two letters are consonants
        
    #Number of vowels >=3 usually for girls, assign more weight
    #v[B-nef+2] = 2 if np.sum(list(map(baby.lower().count, "aeiou")))>=3 else 0
        
    #ends with vowel
    #v[B-nef+1]= 2 if baby[-1] in "aeiouy" else 0
        
    #does not end with vowel
    #v[B-nef+2]= 2 if baby[-1] not in "aeiou" else 0
        
    #begins with consonant and ends with vowel
    #v[B-nef+3]= 1 if baby.isalpha() and baby[0] not in "aeiou" and baby[-1] in "aeiou" else 0
        
    #begins with vowel and ends with consonant
    #v[B-nef+4]= 2 if baby.isalpha() and baby[0] in "aeiou" and baby[-1] not in "aeiou" else 0
    
    
        
    return v

def hashfeatures2(baby, B, FIX):
    v = np.zeros(B)
    #extra features location
    #ef_loc=FIX*26
    ef_loc=2*FIX*26
    baby=baby.lower()
    
    for m in range(FIX):
        #Start hashing characters from right
        v[m*26 + (ord(baby[-(m+1)]) - ord('a'))]=2
    
    for m in range(FIX):
        #Start hashing characters from left
        v[(m+FIX)*26 + (ord(baby[m]) - ord('a'))]=1
    
    #Check the last char - vowel or y
    v[ef_loc]= 1 if baby[-1] in "aeiouy"  else 0
    
    #Check if last two letters are vowels
    v[ef_loc+1]= 1 if baby[-2] in "aeiou" else 0
    
    #Number of vowels >=3 usually for girls, assign more weight
    #v[ef_loc+2] = 1 if np.sum(list(map(baby.lower().count, "aeiou")))>=2 else 0
    
    #begins with consonant and ends with vowel
    #v[ef_loc+2]= 1 if baby.isalpha() and baby[0] not in "aeiou" and baby[-1] in "aeiou" else 0
    
    return v



def hashfeatures3(baby, B, FIX):
    v = np.zeros(B)
    #number of extra features
    
    
    
    baby=baby.lower()
    
    v[ ( ord(baby[-1]) - ord('a') ) % 26] = 1
    v[ ( ord(baby[-2]) - ord('a') ) % 26] = v[ ( ord(baby[-2]) - ord('a') ) % 26] + 1
    
    #last char vowel
    #Check the last char - vowel or y
    v[27]= 1 if baby[-1] in "aeiouy"  else 0
    v[27]= v[27] + (1 if baby[-2] in "aeiou"  else 0)
    
    return v

def name2features2(filename, B=20000, FIX=5, LoadFile=True):
    """
    Output:
    X : n feature vectors of dimension B, (nxB)
    """
    # read in baby names
    if LoadFile:
        with open(filename, 'r') as f:
            babynames = [x.rstrip() for x in f.readlines() if len(x) > 0]
    else:
        babynames = filename.split('\n')
    #Increasing training data set
    #babynames=babynames
    
    n = len(babynames)
    X = np.zeros((n, B))
    for i in range(n):
        X[i,:] = hashfeatures(babynames[i], B, FIX)
        #X[i,:] = hashfeatures3(babynames[i], B, FIX)
        
        
    return X

#</GRADED>

In [42]:
def genTrainFeaturesComp(dimension=128, fix=3):
    """
    function [x,y]=genTrainFeatures
    
    This function calls the python script "name2features.py" 
    to convert names into feature vectors and loads in the training data. 
    
    
    Output: 
    x: n feature vectors of dimensionality d [d,n]
    y: n labels (-1 = girl, +1 = boy)
    """
    
    # Load in the data
    Xgirls = name2features2("girls.train", B=dimension, FIX=fix)
    Xboys = name2features2("boys.train", B=dimension, FIX=fix)
    X = np.concatenate([Xgirls, Xboys])
    
    # Generate Labels
    Y = np.concatenate([-np.ones(len(Xgirls)), np.ones(len(Xboys))])
    
    # shuffle data into random order
    ii = np.random.permutation([i for i in range(len(Y))])
    
    return X[ii, :], Y[ii]

<h4>Credits</h4>
  Parts of this webpage were copied from or heavily inspired by John DeNero's and Dan Klein's (awesome) <a href="http://ai.berkeley.edu/project_overview.html">Pacman class</a>. The name classification idea originates from <a href="http://nickm.com">Nick Montfort</a>.

In [43]:
#changed B
baby="Jahanvi"
a=list(map(baby.lower().count, "aeiou"))
print(*a)
np.sum(a)
print(1 if baby[-1].lower() not in "aeiou" else 0)
print(1 if baby.isalpha() and baby[0] not in "aeiou" and baby[-1] in "aeiou" else 0)

2 0 1 0 0
0
1


In [44]:
DIMS = 150
print('Loading data ...')
X,Y = genTrainFeaturesComp(DIMS)
print('Training classifier ...')
w,b=naivebayesCL(X,Y)
error = np.mean(classifyLinear(X,w,b) != Y)
print('Training error: %.2f%%' % (100 * error))

Loading data ...
Training classifier ...
Training error: 18.75%


In [4]:
babynames =['JK','SN','AK']
np.repeat(babynames,2)
babynames*2


['JK', 'SN', 'AK', 'JK', 'SN', 'AK']

In [1]:
a="jahanvi"
ord('v') - ord('i')
len(a)


7